In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [79]:
train_df = pd.read_csv('data/train ori.csv')
test_df = pd.read_csv('data/test ori.csv')

In [80]:
train_df['waktu_setempat'] = pd.to_datetime(train_df['waktu_setempat']).dt.tz_localize(None)
test_df['waktu_setempat'] = pd.to_datetime(test_df['waktu_setempat']).dt.tz_localize(None)

In [81]:
def interpolate_hourly(group):   
    group = group.set_index('waktu_setempat')
    group = group.resample('1H').first()
    
    # Linearly interpolate the 'rerata_kecepatan' column
    group['rerata_kecepatan'] = group['rerata_kecepatan'].interpolate(method='linear')
    
    # Reset index and forward fill non-'rerata_kecepatan' columns
    group = group.reset_index()
    group[['id_jalan', 'id_titik_mulai', 'id_titik_akhir']] = group[['id_jalan', 'id_titik_mulai', 'id_titik_akhir']].ffill()
    
    return group

# Apply the interpolation function to each group
train_df = train_df.groupby(['id_jalan', 'id_titik_mulai', 'id_titik_akhir']).apply(interpolate_hourly)

# Reset the index
train_df.reset_index(drop=True, inplace=True)

In [82]:
train_df['id_jalan'] = train_df['id_jalan'].astype(int)
train_df['id_titik_mulai'] = train_df['id_titik_mulai'].astype(int)
train_df['id_titik_akhir'] = train_df['id_titik_akhir'].astype(int)

In [83]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492157 entries, 0 to 492156
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   waktu_setempat    492157 non-null  datetime64[ns]
 1   id_jalan          492157 non-null  int32         
 2   id_titik_mulai    492157 non-null  int32         
 3   id_titik_akhir    492157 non-null  int32         
 4   rerata_kecepatan  492157 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int32(3)
memory usage: 13.1 MB


In [84]:
# Remove outliers from 'rerata_kecepatan' column
Q1 = train_df['rerata_kecepatan'].quantile(0.25)
Q3 = train_df['rerata_kecepatan'].quantile(0.75)

IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

train_df = train_df[(train_df['rerata_kecepatan'] >= lower_bound) & (
    train_df['rerata_kecepatan'] <= upper_bound)]

In [85]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 477337 entries, 0 to 492156
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   waktu_setempat    477337 non-null  datetime64[ns]
 1   id_jalan          477337 non-null  int32         
 2   id_titik_mulai    477337 non-null  int32         
 3   id_titik_akhir    477337 non-null  int32         
 4   rerata_kecepatan  477337 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int32(3)
memory usage: 16.4 MB


In [86]:
train_df['unique_id'] = train_df['id_jalan'].astype(str) + '_' + train_df['id_titik_mulai'].astype(str) + '_' + train_df['id_titik_akhir'].astype(str)
test_df['unique_id'] = test_df['id_jalan'].astype(str) + '_' + test_df['id_titik_mulai'].astype(str) + '_' + test_df['id_titik_akhir'].astype(str)

In [87]:
# import pandas as pd
# import seaborn as sns
# # import matplotlib.pyplot as plt

# # Assuming you have already loaded the DataFrame as 'train_df'
# sns.set(style="whitegrid")
# plt.figure(figsize=(12, 8))
# sns.heatmap(train_df.corr(), annot=True, cmap="coolwarm", fmt=".2f")
# plt.title("Correlation Heatmap for train_df")
# plt.show()

In [88]:
train_df = train_df.rename(columns={'waktu_setempat': 'ds', 'rerata_kecepatan': 'y'})
test_df = test_df.rename(columns={'waktu_setempat': 'ds'})

In [89]:
columns_to_drop = ['id_jalan', 'id_titik_mulai', 'id_titik_akhir']
train_df.drop(columns=columns_to_drop, inplace=True)
test_df.drop(columns=columns_to_drop, inplace=True)

In [90]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 477337 entries, 0 to 492156
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   ds         477337 non-null  datetime64[ns]
 1   y          477337 non-null  float64       
 2   unique_id  477337 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 14.6+ MB


In [91]:
train_df['unique_id'] = train_df['unique_id'].astype('category')
test_df['unique_id'] = test_df['unique_id'].astype('category')

In [92]:
id_series = test_df['id']
id_df = pd.DataFrame({'id': id_series})

In [93]:
import pandas as pd
import numpy as np
from prophet import Prophet

fitted_models = {}
# Initialize an empty list to store validation forecasted DataFrames
validation_forecasted_dfs = []

# Group the train data by unique_id and fit a model for each group
for unique_id, group_data in train_df.groupby('unique_id'):
    model = Prophet()
    
    # Split the group_data into train and validation sets
    train_size = int(0.8 * len(group_data))
    train_group_data = group_data[:train_size]
    val_group_data = group_data[train_size:]
    
    # Fit the model on the training set
    model.fit(train_group_data[['ds', 'y']])
    
    # Create a DataFrame with the future dates for validation
    future_val = model.make_future_dataframe(periods=len(val_group_data), freq='H', include_history=False)
    
    # Make validation forecasts
    forecast_val = model.predict(future_val)
    
    # Append validation forecasted results to the list
    validation_forecasted_dfs.append(
        forecast_val.assign(unique_id=unique_id, y=val_group_data['y'].values)
    )

    fitted_models[unique_id] = model

# Concatenate all validation forecasted DataFrames into one
validation_forecasted_results = pd.concat(validation_forecasted_dfs, ignore_index=True)

# Calculate SMAPE for the validation forecasts
def calculate_smape(actual, forecasted):
    absolute_diff = np.abs(forecasted - actual)
    denominator = (np.abs(actual) + np.abs(forecasted)) / 2
    smape = 100 * np.nanmean(2 * absolute_diff / denominator)
    return smape

smape_list = []

for unique_id, group_data in validation_forecasted_results.groupby('unique_id'):
    smape = calculate_smape(group_data['y'], group_data['yhat'])
    smape_list.append(smape)

mean_smape = np.mean(smape_list)

print(f"Mean SMAPE for validation forecasts: {mean_smape:.2f}")


09:59:49 - cmdstanpy - INFO - Chain [1] start processing
09:59:49 - cmdstanpy - INFO - Chain [1] done processing
09:59:49 - cmdstanpy - INFO - Chain [1] start processing
09:59:49 - cmdstanpy - INFO - Chain [1] done processing
09:59:49 - cmdstanpy - INFO - Chain [1] start processing
09:59:49 - cmdstanpy - INFO - Chain [1] done processing
09:59:49 - cmdstanpy - INFO - Chain [1] start processing
09:59:49 - cmdstanpy - INFO - Chain [1] done processing
09:59:49 - cmdstanpy - INFO - Chain [1] start processing
09:59:49 - cmdstanpy - INFO - Chain [1] done processing
09:59:50 - cmdstanpy - INFO - Chain [1] start processing
09:59:50 - cmdstanpy - INFO - Chain [1] done processing
09:59:50 - cmdstanpy - INFO - Chain [1] start processing
09:59:50 - cmdstanpy - INFO - Chain [1] done processing
09:59:50 - cmdstanpy - INFO - Chain [1] start processing
09:59:50 - cmdstanpy - INFO - Chain [1] done processing
09:59:50 - cmdstanpy - INFO - Chain [1] start processing
09:59:50 - cmdstanpy - INFO - Chain [1]

Mean SMAPE for validation forecasts: 19.71


In [94]:
# Initialize an empty list to store test forecasted DataFrames
test_forecasted_dfs = []

# Use fitted models to make predictions for each unique_id in the test DataFrame
for unique_id, group_data in test_df.groupby('unique_id'):
    if unique_id in fitted_models:
        print(f"Making forecast for unique_id: {unique_id}")
        model = fitted_models[unique_id]

        # Create a DataFrame with the future dates for the unique_id
        future_test = model.make_future_dataframe(periods=len(group_data), freq='H', include_history=False)

        # Make predictions for the test dataset
        forecast_test = model.predict(future_test)

        # Append test forecasted results to the list
        test_forecasted_dfs.append(
            forecast_test.assign(unique_id=unique_id)
        )
    else:
        print(f"Skipping unique_id: {unique_id} (not in fitted_models)")

# Concatenate all test forecasted DataFrames into one
test_forecasted_results = pd.concat(test_forecasted_dfs, ignore_index=True)


Making forecast for unique_id: 1210_1030634572_1030634588
Making forecast for unique_id: 1210_1030634572_1663451615
Making forecast for unique_id: 1210_1030634588_1030634572
Making forecast for unique_id: 1210_1030634588_1701556095
Making forecast for unique_id: 1210_1131002278_1663421251
Making forecast for unique_id: 1210_1131002278_1663475470
Making forecast for unique_id: 1210_16385387_1663432888
Making forecast for unique_id: 1210_16385387_1663442661
Making forecast for unique_id: 1210_1663386752_195849
Making forecast for unique_id: 1210_1663386752_195850
Making forecast for unique_id: 1210_1663421251_1131002278
Making forecast for unique_id: 1210_1663421251_1663461749
Making forecast for unique_id: 1210_1663426986_1663475339
Making forecast for unique_id: 1210_1663426986_480041
Making forecast for unique_id: 1210_1663432888_16385387
Making forecast for unique_id: 1210_1663432888_480041
Making forecast for unique_id: 1210_1663442661_16385387
Making forecast for unique_id: 1210_16

In [95]:
train_df['unique_id'].value_counts()

unique_id
50841260_-2147483648_-2147483648     8342
4061596_-2147483648_-2147483648      7376
142479648_-2147483648_-2147483648    4145
4060819_-2147483648_-2147483648      2107
24704286_-2147483648_-2147483648     2106
                                     ... 
97453767_20961235_312670730           151
691007296_43412521_43412859           144
182210371_33139292_1587291223         139
8504977_-2147483648_2378878           133
4061596_1237102032_227758              56
Name: count, Length: 862, dtype: int64

In [96]:
test_forecasted_results

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,unique_id
0,2020-02-18 15:00:00,40.455177,28.030525,40.005596,40.455177,40.455177,-6.870536,-6.870536,-6.870536,-6.385384,-6.385384,-6.385384,-0.485153,-0.485153,-0.485153,0.0,0.0,0.0,33.584641,1210_1030634572_1030634588
1,2020-02-18 16:00:00,40.460758,26.959148,38.702261,40.460758,40.460758,-7.442604,-7.442604,-7.442604,-6.937625,-6.937625,-6.937625,-0.504979,-0.504979,-0.504979,0.0,0.0,0.0,33.018154,1210_1030634572_1030634588
2,2020-02-18 17:00:00,40.466339,27.445999,39.130364,40.466339,40.466339,-7.038759,-7.038759,-7.038759,-6.512360,-6.512360,-6.512360,-0.526399,-0.526399,-0.526399,0.0,0.0,0.0,33.427580,1210_1030634572_1030634588
3,2020-02-18 18:00:00,40.471920,28.772785,40.966552,40.471824,40.471920,-5.739467,-5.739467,-5.739467,-5.189699,-5.189699,-5.189699,-0.549768,-0.549768,-0.549768,0.0,0.0,0.0,34.732453,1210_1030634572_1030634588
4,2020-02-18 19:00:00,40.477501,30.498639,42.275600,40.477199,40.477737,-3.864560,-3.864560,-3.864560,-3.289169,-3.289169,-3.289169,-0.575392,-0.575392,-0.575392,0.0,0.0,0.0,36.612940,1210_1030634572_1030634588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88447,2020-02-24 14:00:00,22.940913,17.484743,25.003003,22.939917,22.942003,-1.714783,-1.714783,-1.714783,-1.359874,-1.359874,-1.359874,-0.354909,-0.354909,-0.354909,0.0,0.0,0.0,21.226130,97453767_33203791_20961350
88448,2020-02-24 15:00:00,22.943557,17.067882,24.769909,22.942556,22.944664,-1.943905,-1.943905,-1.943905,-1.544905,-1.544905,-1.544905,-0.399000,-0.399000,-0.399000,0.0,0.0,0.0,20.999652,97453767_33203791_20961350
88449,2020-02-24 16:00:00,22.946201,16.046680,23.585378,22.945195,22.947325,-3.068103,-3.068103,-3.068103,-2.638426,-2.638426,-2.638426,-0.429677,-0.429677,-0.429677,0.0,0.0,0.0,19.878098,97453767_33203791_20961350
88450,2020-02-24 17:00:00,22.948846,14.528969,22.128750,22.947827,22.949981,-4.551138,-4.551138,-4.551138,-4.104487,-4.104487,-4.104487,-0.446652,-0.446652,-0.446652,0.0,0.0,0.0,18.397707,97453767_33203791_20961350


In [97]:
results = pd.concat([id_df, test_forecasted_results['yhat']], axis=1)
results.rename(columns={'yhat': 'rerata_kecepatan'}, inplace=True)

print(results)

            id  rerata_kecepatan
0            0         33.584641
1            1         33.018154
2            2         33.427580
3            3         34.732453
4            4         36.612940
...        ...               ...
127484  127484               NaN
127485  127485               NaN
127486  127486               NaN
127487  127487               NaN
127488  127488               NaN

[127489 rows x 2 columns]


In [98]:
# Store the concatenated DataFrame to a CSV file
results.to_csv('Data/results.csv', index=False)

print("Data saved")

Data saved


In [99]:
# # Create a DataFrame with the future dates you want to forecast (for test)
# future_test = model.make_future_dataframe(periods=len(test_df), freq='H', include_history=False)

# # Make predictions for the test dataset
# forecast_test = model.predict(future_test)

# # Join forecasted values with the actual values in the test_df
# forecasted_test_df = test_df.join(forecast_test.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']])
# forecasted_test_df


In [100]:
# id_series = test_df['id']
# id_df = pd.DataFrame({'id': id_series})

# test_df.drop(columns=['id'], inplace=True)
# # test_df.drop(columns=['id', 'lanes', 'lanes_forward', 'lanes_backward'], inplace=True)
# # test_df.drop(columns=['id', 'busway', 'lanes_forward', 'lanes_backward'], inplace=True)